In [ ]:
pip install -r requirements.txt

In [40]:
import os
import xml.etree.cElementTree as et

In [41]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
from PIL import Image
import tensorflow as tf
import tensorflow_hub as hub


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense,Conv2D , MaxPooling2D, Flatten,BatchNormalization,Dropout

from sklearn.metrics import classification_report , confusion_matrix , accuracy_score , auc
from sklearn.model_selection import train_test_split

import cv2

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D

In [42]:
""" Unzipping data set to a local director  """

from zipfile import ZipFile

with ZipFile("F:\\Coding\\cv_challenge\\archive.zip", 'r') as Dataset:
    Dataset.extractall()

In [ ]:
# F:\Coding\cv_challenge\NEU-DET\train\images\crazing
data_folder = "F:\\Coding\\cv_challenge\\NEU-DET"

train_folder = os.path.join(data_folder, "train", "images", )
train_images_subfolders = os.listdir(train_folder)
train_image_files = []
for subfolder in train_images_subfolders:
    train_image_path = os.path.join(data_folder, "train", "images", subfolder)
    train_image_files += os.listdir(train_image_path)
    print(len(train_image_files))
    # image = Image.open(train_image_path + "\\" + train_image_files[0])
    # image.show()
    print(train_image_path)

train_anno_path = os.path.join(data_folder,"train", "annotations")
train_anno_files = os.listdir(train_anno_path)
# print(train_anno_files)

In [44]:
""" Extracting Lables from XML files """
def parseXML(xmlfile):
    # create element tree object 
    tree = et.parse(xmlfile)  
    # get root element 
    root = tree.getroot()

    for name in root.iter('name'):
        label = (name.text)

    return label

In [48]:
"""
Get distinct class labels from xml files
Returns a list of distinct labels
"""

labels = []
for anno in train_anno_files:
    label = parseXML(train_anno_path +"\\"+anno)
    if label not in labels:
        labels.append(label)
print(labels)


['crazing', 'patches', 'inclusion', 'pitted_surface', 'rolled-in_scale', 'scratches']


In [51]:
"""
Assigning distinct values to each distinct label
"""
image_labels = {name: index for index, name in enumerate(labels)}
print(image_labels)

{'crazing': 0, 'patches': 1, 'inclusion': 2, 'pitted_surface': 3, 'rolled-in_scale': 4, 'scratches': 5}


In [38]:
""" Makeing corresponding image-label pairs """

for img in train_image_files[:10]:
    base_img, extn = img.split(".")
    print(base_img)
    

crazing_1
crazing_10
crazing_100
crazing_101
crazing_102
crazing_103
crazing_104
crazing_105
crazing_106
crazing_107


In [91]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [102]:
i1 = "F:\\Coding\\cv_challenge\\NEU-DET\\train\\images\\inclusion\\inclusion_1.jpg"
anno1 = "F:\\Coding\\cv_challenge\\NEU-DET\\train\\annotations\\inclusion_1.xml"
image_string = open(i1, 'rb').read()
label = None

tree = et.parse(anno1)  
root = tree.getroot()

for name in image_labels:
  if name in i1:
    label =  image_labels[name]

filename = root.find("filename").text
width = int(root.find("size/width").text)
height = int(root.find("size/height").text)

xmin = float(root.find("object/bndbox/xmin").text)
ymin = float(root.find("object/bndbox/ymin").text)
xmax = float(root.find("object/bndbox/xmax").text)
ymax = float(root.find("object/bndbox/ymax").text)

label_text = root.find("object/name").text
label = image_labels.get(label_text, -1)  # Get label index or -1 if not found
    

# Create a dictionary with features that may be relevant.
def image_example(image_string, label):
  image_shape = tf.io.decode_jpeg(image_string).shape
  feature = {
        "image/width": _int64_feature(width),
        "image/height": _int64_feature(height),
        "image/filename":_bytes_feature(bytes(filename, 'utf-8')),
        # 'image/source_id': root.find("source").text,
        "image/encodedrawdata": _bytes_feature(image_string),
        "image/format": _bytes_feature(bytes(filename.split(".")[-1], 'utf-8')),  # Assuming filename contains format
        "image/object/bbox/xmin": _float_feature(xmin),
        "image/object/bbox/xmax": _float_feature(xmax),
        "image/object/bbox/ymin": _float_feature(ymin),
        "image/object/bbox/ymax": _float_feature(ymax),
        "image/object/class/text": _bytes_feature(bytes(label_text, 'utf-8')),
        "image/object/class/label": _int64_feature(label),
        "image/object/class/single": _int64_feature(label),  # Assuming single class per object
        "image/object/difficult": _int64_feature(0),  # Difficult is 0
        "image/object/truncated": _int64_feature(0),  # Truncated is 0
        "image/object/view": _bytes_feature(bytes("Unspecified", 'utf-8'))  # Assuming view is Unspecified for simplicity
    }

  return tf.train.Example(features=tf.train.Features(feature=feature))

for line in str(image_example(image_string, label)).split('\n')[:15]:
  print(line)
print('...')

# print(str(image_example(image_string, label)))
"""
Followed the article of Reading-Writing TFRecord for Images
https://www.tensorflow.org/tutorials/load_data/tfrecord
"""

features {
  feature {
    key: "image/width"
    value {
      int64_list {
        value: 200
      }
    }
  }
  feature {
    key: "image/object/view"
    value {
      bytes_list {
        value: "Unspecified"
      }
...


' Followed the article of Reading-Writing TFRecord for Images'

In [11]:
def create_fn(img):
  feature = {
        "image/width": tf.io.FixedLenFeature([], tf.int64),
        "image/height": tf.io.FixedLenFeature([], tf.int64),
        'image/filename': tf.io.FixedLenFeature([], tf.string),
        'image/source_id': tf.io.FixedLenFeature([], tf.string),
        'image/encodedrawdata': tf.io.FixedLenFeature([], tf.string),
        'image/format': tf.io.FixedLenFeature([], tf.string),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/text': tf.io.VarLenFeature(tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
        'image/object/class/single': tf.io.FixedLenFeature([], tf.int64),
        'image/object/difficult': tf.io.VarLenFeature(tf.int64),
        'image/object/truncated': tf.io.VarLenFeature(tf.int64),
        'image/object/view': tf.io.VarLenFeature(tf.string)
    }
  tf_example = tf.train.Example(features=tf.train.Features(feature = feature))
  return tf_example

In [12]:
output_path = "F:\\Coding\\cv_challenge\\TF-Data\\Tf_train"
record_file = 'images.tfrecords'
# writer = tf.io.TFRecordWriter(output_path + record_file)

  # for example in examples:
tf_example = create_fn("F:\\Coding\\cv_challenge\\NEU-DET\\train\\images\\crazing\\crazing_1.jpg")
# writer.write(tf_example.SerializeToString())

# writer.close()

TypeError: Feature.CopyFrom() takes exactly one argument (3 given)